In [3]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
os.listdir("results")

['drug_cells_kernels_best_parameters.csv',
 'test02_merged_fitted_sigmoid4_123_with_drugs_properties.csv',
 'drug_cells_PubChem_scaled_kernels_best_parameters.csv',
 'kernel_learning_01_2.csv',
 '.DS_Store',
 'kernel_learning_03_2.csv',
 'test02_merged_fitted_sigmoid4_123_with_drugs_description.csv',
 'drug_features_with_pubchem_properties.csv',
 'kernel_learning_03.csv',
 'kernel_learning_02.csv',
 'kernel_ridge_model_parameters_drug_cells_PubChem.csv',
 'merged_fitted_sigmoid4_123_with_drugs_description.csv',
 'filtered_drug_profiles_13.csv',
 'merged_fitted_sigmoid4_123_with_drugs_properties_split_target.csv',
 'kernel_learning_01.csv',
 'filtered_drug_profiles_12.csv',
 'svr_model_parameters_drug_cells_PubChem_Scaling.csv',
 'merged_fitted_sigmoid4_123_with_drugs_description_split_target.csv',
 'statistics_of_sigmoid_coefficients.csv',
 'kernel_ridge_model_parameters_drug_cells_PubChem_Scaling.csv',
 'kernel_learning_04.csv',
 'svr_model_parameters_drug_cells_description.csv',
 'dr

In [5]:
train_123 = pd.read_csv("results/train08_merged_fitted_sigmoid4_123_with_drugs_properties.csv").drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [6]:
train_123.columns[-20:]

Index(['3bonds', 'Mitosis', 'WNT signaling', 'bond_stereo_count', 'complexity',
       'P', 'H', 'Cl', 'rotatable_bond_count', 'undefined_atom_stereo_count',
       'B', 'defined_atom_stereo_count', 'molecular_formula', 'second_target',
       'Other, kinases', 'Metabolism', 'JNK and p38 signaling', 'Drug_Name',
       'Cytoskeleton', 'MAX_CONC'],
      dtype='object')

### Average sigmoid

In [7]:
norm_response  = ["norm_cells_"+str(i) for i in range(10)]
conc_columns  = ["fd_num_"+str(i) for i in range(10)]

col_not_to_use = ["Unnamed: 0", "Unnamed: 0.1", 'DRUG_ID', 'COSMIC_ID', 'Drug_Name', 
                  "Synonyms", "Target", "deriv_found", "PubChem_ID","elements", "inchi_key",
                  "canonical_smiles", "inchi_string", "molecular_formula",
                  "third_target", "first_target", "second_target", "Target_Pathway"]

columns_to_use = ['DRUG_ID', 'COSMIC_ID', 'Drug_Name'] + list(set(train_123.columns)
                                                              -set(norm_response)-set(conc_columns)
                                                              -set(col_not_to_use))
train = np.zeros([1, len(columns_to_use)+2])
for i in range(10):
    train = np.vstack((train, train_123[columns_to_use + ["fd_num_"+str(i), "norm_cells_"+str(i)]].values))

In [8]:
train.shape, train_123.shape

((18171, 1132), (1817, 1154))

In [9]:
train_df_full = pd.DataFrame(data= train[1:,:], columns = columns_to_use + ["x_conc", "y_response"])
train_df_full

,DRUG_ID,COSMIC_ID,Drug_Name,chr12:103889148-103889843(C12orf42)_HypMET,BAZ2B_mut,RBM5_mut,MAGI2_mut,loss:cnaPANCAN114 (CIITA),gain:cnaPANCAN128,chr19:51069359-51069600(LRRC4B)_HypMET,...,loss:cnaPANCAN10 (CIC),loss:cnaPANCAN228,chr5:157002174-157003182(ADAM19)_HypMET,loss:cnaPANCAN92,chr11:66247269-66248249(DPP3)_HypMET,chr5:63461448-63462106()_HypMET,HNF1A_mut,ACVR2A_mut,x_conc,y_response
0,11,1322218,Paclitaxel,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,11,753614,Paclitaxel,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,11,1330947,Paclitaxel,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,11,1330931,Paclitaxel,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,11,906841,Paclitaxel,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18165,1149,908473,TW 37,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0.26221
18166,1149,688025,TW 37,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.142445
18167,1149,687794,TW 37,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0.25525
18168,1149,1290806,TW 37,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.15463


In [1]:
from sklearn.svm import SVR
from sklearn.linear_model import GammaRegressor

ImportError: cannot import name 'GammaRegressor'